<div class="alert alert-block alert-info">
<b>Deadline:</b> May 11, 2022 (Wednesday) 23:00
</div>


# Exercise 2. Conditional generation with PixelCNN

The goal of this exercise is to do conditional generation with the PixelCNN model.
The basic idea of the conditioning is described in Section 2.3 of [this paper](https://arxiv.org/pdf/1606.05328.pdf). However, we will use a much simpler model.

In [1]:
skip_training = True  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [3]:
import time

import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools
import tests

In [4]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [5]:
# Select the device for training (use GPU if you have one)
device = torch.device('cpu')
#device = torch.device('cuda:0')

In [6]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device('cpu')

# Data

In this exercise, we use standard MNIST data.

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

In [8]:
images, labels = iter(trainloader).next()
tools.plot_images(images[:8], ncol=4, cmap=plt.cm.Greys, clim=[0,1])

# Masked convolutional layer

In the cell below, copy the implementation of the `MaskedConv2d` from the PixelCNN notebook.

In [9]:
class MaskedConv2d(nn.Module):
    # YOUR CODE HERE
    def __init__(self, in_channels, out_channels, kernel_size, blind_center=False):
        """
        Args:
          in_channels (int): Number of input channels.
          out_channels (int): Number of output channels.
          kernel_size (int): Kernel size similar to nn.Conv2d layer.
          blind_center (bool): If True, the kernel has zero in the center.
        """
        super(MaskedConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=int((kernel_size-1)/2), bias = False)

        self.register_buffer('mask', self.conv.weight.data.clone())
        
        #print(self.conv.weight.size())
        width = self.conv.weight.size(2)
        w_index = width//2
        height = self.conv.weight.size(3)
        h_index = height//2
        
        self.mask.fill_(1)
        if blind_center==True:
            self.mask[:, :, h_index + 1:] = 0
            self.mask[:, :, h_index, w_index:] = 0
        else:
            self.mask[:, :, h_index + 1:] = 0
            self.mask[:, :, h_index , w_index + 1:] = 0 
        #print(self.mask)
        #raise NotImplementedError()

    def forward(self, x):
        """
        Args:
          x of shape (batch_size, in_channels, height, width): Input images.
        
        Returns:
          y of shape (batch_size, out_channels, height, width): Output images.
        """
        # YOUR CODE HERE
        
        self.conv.weight.data = self.conv.weight.data * self.mask
        y = self.conv(x)
        #print(y.size())
        return y   
    
    #raise NotImplementedError()

# Conditional PixelCNN

Conditional PixelCNN models allows to generate images of a desired class. This can be achieved by providing the desired class label to every layer of the PixelCNN model. In this notebook, we do it in the following way: the input of each masked convolutional layer is:
$$\mathbf{x} + \mathbf{W} \mathbf{h}$$
where
  * $\mathbf{x}$ is the output of the previous layer
  * $\mathbf{h}$ is a 10-dimensional one-hot coded vector of the desired class
  * $\mathbf{W}$ is $c \times 10$ matrix (parameter of a fully-connected layer), where $c$ is the number of input channels in the masked convolutional layer.

You need to implement the conditional PixelCNN model in the cell below.

Notes:
* Use an architecture *similar* to the PixelCNN architecture in the first notebook. The architecture is not exactly same because of the extra inputs $\mathbf{h}$. Please use the same kernel sizes as in the first notebook because we test the receptive fields.
* The parameters $\mathbf{W}$ are not shared across layers. Thus, you need as many parameters $\mathbf{W}$ as you have masked convolutional layers.

In [10]:
class ConditionalPixelCNN(nn.Module):
    def __init__(self, n_channels=64, kernel_size=7):
        """PixelCNN model for conditional generation."""
        # YOUR CODE HERE
        super(ConditionalPixelCNN, self).__init__()
        
        self.conv0 = MaskedConv2d(1, n_channels, kernel_size, blind_center=True)
        self.nor0 = nn.BatchNorm2d(n_channels)
        self.relu0 = nn.ReLU()
        
        # 7 blocks
        self.conv1 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor1 = nn.BatchNorm2d(n_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor2 = nn.BatchNorm2d(n_channels)
        self.relu2 = nn.ReLU()
        
        self.conv3 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor3 = nn.BatchNorm2d(n_channels)
        self.relu3 = nn.ReLU()
        
        self.conv4 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor4 = nn.BatchNorm2d(n_channels)
        self.relu4 = nn.ReLU()

        self.conv5 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor5 = nn.BatchNorm2d(n_channels)
        self.relu5 = nn.ReLU()
        
        self.conv6 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor6 = nn.BatchNorm2d(n_channels)
        self.relu6 = nn.ReLU()
        
        self.conv7 = MaskedConv2d(n_channels, n_channels, kernel_size, blind_center=False)
        self.nor7 = nn.BatchNorm2d(n_channels)
        self.relu7 = nn.ReLU()
        
        # final 1x1 convolution
        self.conv8 = nn.Conv2d(n_channels, 256, kernel_size=1)
        
        self.fc = nn.Linear(10, 1)
       
        #raise NotImplementedError()
        
    def forward(self, x, labels):
        """Compute logits of the conditional probabilities p(x_i|x_1, ..., x_{i-1}) of the PixelCNN model.
        
        Args:
          x of shape (batch_size, 1, 28, 28): Tensor of input images.
          labels of shape (batch_size): Long tensor of the desired classes of the generated samples.
        
        Returns:
          logits of shape (batch_size, 256, 28, 28): Tensor of logits of the conditional probabilities
                                                      for each pixel.
        
        NB: Do not use softmax nonlinearity after the last layer.
        """
        # YOUR CODE HERE
        onehot = torch.FloatTensor(labels.shape[0], 10).zero_().to(labels.device)
        onehot.scatter_(1, labels.unsqueeze(1), 1)
        b = self.fc(onehot).unsqueeze(2).unsqueeze(3).repeat(1, 1, x.shape[-2], x.shape[-1])
        
        x0 = self.relu0(self.nor0(self.conv0(x+b)))
        
        x1 = self.relu1(self.nor1(self.conv1(x0+b)))
        x2 = self.relu2(self.nor2(self.conv2(x1+b)))
        x3 = self.relu3(self.nor3(self.conv3(x2+b)))
        x4 = self.relu4(self.nor4(self.conv4(x3+b)))
        x5 = self.relu5(self.nor5(self.conv5(x4+b)))
        x6 = self.relu6(self.nor6(self.conv6(x5+b)))
        x7 = self.relu7(self.nor7(self.conv7(x6+b)))
        
        x8 = self.conv8(x7+b)
        
        logits = x8
        
        return logits
        
        raise NotImplementedError()

In [11]:
def test_ConditionalPixelCNN_shapes():
    net = ConditionalPixelCNN(n_channels=64, kernel_size=7)

    batch_size = 2
    x = torch.ones(batch_size, 1, 28, 28)
    labels = torch.zeros(batch_size, dtype=torch.long)
    y = net(x, labels)
    assert y.shape == torch.Size([batch_size, 256, 28, 28]), f"Bad y.shape: {y.shape}"
    print('Success')

test_ConditionalPixelCNN_shapes()

Success


### Loss function for training conditional PixelCNN

The `loss_fn()` function is identical to the `loss_fn()` from the PixelCNN notebook.

In [12]:
def loss_fn(logits, x):
    """Compute loss of the conditional PixelCNN model. Please see PixelCNN.loss for more details.

    Args:
      logits of shape (batch_size, 256, 28, 28): Logits of the conditional probabilities
                  p(x_i | x_1,...,x_{i-1}) of the 256 intensities of pixel x_i computed using all
                  previous pixel value x_1,...,x_{i-1}.
      x of shape (batch_size, 1, 28, 28): Images used to produce `generated_x`. The values of pixel
                  intensities in x are between 0 and 1.

    Returns:
      loss: Scalar tensor which contains the value of the loss.
    """
    # YOUR CODE HERE
    y = (x * 255).long().squeeze(1)
    loss = F.cross_entropy(logits.to(device), y)
    
    return loss
    raise NotImplementedError()

### Generation procedure

The `generate()` function is *almost* identical to the `generate()` function from the PixelCNN notebook. It additionally receives the labels of the desired classes so that they can be used in the forward computations of the conditional PixelCNN model.

In [13]:
def generate(net, labels, image_size=(28, 28), device='cpu'):
    """Generate samples using a trained conditional PixelCNN model.
    Note: use as device labels.device.

    Args:
      net: Conditional PixelCNN model.
      labels of shape (n_samples): Long tensor of the desired classes of the generated samples.
      image_size: Tuple of image size (height, width).
      device:     Device to use.
    
    Returns:
      samples of shape (n_samples, 1, height, width): Generated samples.
    """
    # YOUR CODE HERE
    with torch.no_grad():
        net.eval()
        h = image_size[0]
        w = image_size[1]
        samples = torch.Tensor(labels.shape[0], 1, h, w).to(device)
        samples.fill_(0)
        for i in range(h):
            for j in range(w):
                output = net(samples, labels)[:, :, i, j]
                samples[:, :, i, j] = torch.multinomial(F.softmax(output), 1).float() / 255.0

    return samples
    raise NotImplementedError()

# Train the model

In [14]:
# Create network
net = ConditionalPixelCNN(n_channels=64, kernel_size=7)
net.to(device)

ConditionalPixelCNN(
  (conv0): MaskedConv2d(
    (conv): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  )
  (nor0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (conv1): MaskedConv2d(
    (conv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  )
  (nor1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): MaskedConv2d(
    (conv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  )
  (nor2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): MaskedConv2d(
    (conv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  )
  (nor3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (conv4): MaskedConv2d(
    (conv): Conv2d(64, 64, kernel_size=(7, 7), 

In [15]:
# Plot generated samples for an untrained model
# Note: generation on CPU may take a significant amount of time
if not skip_training:
    labels = torch.cat([torch.arange(10) for _ in range(12)], dim=0).to(device)
    samples = generate(net, labels, device=device)
    tools.plot_generated_samples(samples, ncol=10)

### Training loop

Implement the training loop in the cell below. The recommended hyperparameters:
* Adam optimizer with learning rate 0.001
* Number of epochs: 11.

Hints:
- The loss values are similar to the loss values in the PixelCNN notebook.
- Please use this code to plot 120 generated samples after each epoch. This will allow you to track the training progress.
```
# Generate samples
with torch.no_grad():
    net.eval()
    labels = torch.cat([torch.arange(10) for _ in range(12)], dim=0).to(device)
    samples = generate(net, labels, device=device)
    tools.plot_generated_samples(samples, ncol=10)
```
- The generated images should be of great quality but you should definitely recognize the desired classes of the digits.
- **Do not forget to set the model into the training mode by `net.train()` before training.**

In [16]:
if not skip_training:
    # YOUR CODE HERE
    import time
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    n_epochs = 11
    plt_loss = []
    for epoch in range(n_epochs):
        time_start=time.time()
        batch_loss = []
        count = 0
        for i, data in enumerate(trainloader):
            net.train()
            images, labels = data
            optimizer.zero_grad()
            out = net(images, labels)
            
            loss = loss_fn(out, images)
            loss.backward()
            batch_loss.append(loss.item()) 
            optimizer.step()
            count += 1
            if count%50 == 0:
                print(count,"/",len(trainloader))
        plt_loss.append(np.mean(batch_loss))
        time_end=time.time()
        print('Epoch ',epoch,' Avg loss: ',plt_loss[-1],"Time: ",time_end-time_start)    
    
    # Generate samples
    with torch.no_grad():
        net.eval()
        labels = torch.cat([torch.arange(10) for _ in range(12)], dim=0).to(device)
        samples = generate(net, labels, device=device)
        tools.plot_generated_samples(samples, ncol=10)
                
        
    
    #raise NotImplementedError()

In [17]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(net, '2_cond_pixelcnn.pth', confirm=False)

In [18]:
if skip_training:
    net = ConditionalPixelCNN(n_channels=64, kernel_size=7)
    tools.load_model(net, '2_cond_pixelcnn.pth', device)

Model loaded from 2_cond_pixelcnn.pth.


### Generate samples

In [19]:
# Save generated samples (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    with torch.no_grad():
        net.eval()
        labels = torch.cat([torch.arange(10) for _ in range(12)], dim=0).to(device)
        samples = generate(net, labels, device=device)
        torch.save(samples, '2_cond_pixelcnn_samples.pth')
else:
    samples = torch.load('2_cond_pixelcnn_samples.pth', map_location=lambda storage, loc: storage)

tools.plot_generated_samples(samples, ncol=10)

In [20]:
# This cell tests the training loss of the trained model

<div class="alert alert-block alert-info">
<b>Conclusion</b>
</div>

In this notebook, we learned how to train a conditional PixelCNN model.